In [ ]:
from brfinance import CVMAsyncBackend
from datetime import datetime, date
from bs4 import BeautifulSoup

import pandas
import logging
import requests
import re
import warnings
import os

warnings.filterwarnings("ignore")

In [ ]:
class CVM():

    def log_file(self):
        """ ARQUIVO DE LOG DE REFERENCIA
            ARMAZENA TODAS AS INFORMACOES DAS RODAGENS DO DIA
            INFORMACOES, WARNINGS, ERRORS E ETC.
            POR DEFAULT O NOME DO ARQUIVO E RELATIVO AO DIA DA EXECUCAO
        """

        self.logger = logging.getLogger("log_file")
        self.logger.setLevel(logging.INFO)
        try:
            file_handler = logging.FileHandler(f"log_files//{str(date.today())}.log")
        except FileNotFoundError:
            os.mkdir("log_files//")
            file_handler = logging.FileHandler(f"log_files//{str(date.today())}.log")
        formatter = logging.Formatter('%(asctime)s - %(levelname)s > %(message)s')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        self.logger.info("Inicializando processamento de dados via Feed CVM")
        self.logger.info(f"By Kevin Pergher - GitHub: @KgPergher98 - Dia: {str(date.today())}")

    def __init__(self) -> None:

        # CRIA ARQUIVO DE LOG
        CVM.log_file(self)

        try:
            # INSTANCIA A CLASSE CMV Async
            self.cvm_http_client = CVMAsyncBackend()
            self.logger.info("CVM Async (BrFinance) instanciada com sucesso")
        except Exception as exc:
            self.logger.error(f"CVM Async (BrFinance) nao instanciada - {exc}")

    def get_cvm_categories(self):
        """ EXTRAI AS INFORMAÇÕES DE CATEGORIAS
            I.E. OS TIPOS DE DOCUMENTOS DISPONÍVEIS

            UTILIZA O BrFINANCE E DADOS DA CVM
        """
        categories = self.cvm_http_client.get_consulta_externa_cvm_categories()
        categories = pandas.DataFrame.from_dict(categories, orient = "index").reset_index()
        categories.columns = ["BrFin", "Description"]
        categories.to_csv("feeder_db//CategoriasDocsCVM.txt", index = False)

    def get_cvm_codes(self):
        """ EXTRAI OS CODIGOS CVM + STATUS CVM

            UTILIZA A BIBLIOTECA BrFINANCE E DADOS DA CVM
        """
        codes = self.cvm_http_client.get_cvm_codes()
        codes = pandas.DataFrame.from_dict(codes, orient = "index")
        pattern = r'\((.*?)\)'
        codes[1] = codes[0].str.extract(pattern).fillna("-")
        codes[0] = codes[0].str.replace(pattern, "", regex = True)
        codes = codes.reset_index()
        codes.columns = ["CVMCode", "FirmName", "CVMStatus"]
        codes.to_csv("feeder_db//CodigosDocsCVM.txt", index = False)

    def format_cnpj(original_number):
        # FORMATACAO DE CNPJ
        if original_number != "-":
            formatted_cnpj = original_number[:2] + '.' + original_number[2:5] + '.' 
            return(formatted_cnpj + original_number[5:8] + '/' + original_number[8:12] + '-' + original_number[12:])
        else:
            return(original_number)

    def firms_informations(self, extract_data:bool = True, fetch_data:bool = True, fetch_br_fin:bool = False, 
                           record_file:str = "feeder_db//EmpresasBrasilCVM", merging_files:bool = True, output_merging:str = "feeder_db//Brazil"):

        """ ADQUIRE INFORMAÇÕES DE COMPANHIAS ABERTAS BRASILEIRAS VIA CVM

            REFERENCIA DE SITE: https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAbOrdAlf.aspx?LetraInicial=3

            EXTRACT_DATA   [BOOL] > EXTRAI OS DADOS EM TABELAS, SEM CABEÇALHO E CRIA ARQUIVO DE REFERENCIA DE DADOS EM FORMATO .TXT
            FETCH_DATA     [BOOL] > AJUSTA DATAFRAME DE REFERÊNCIA CVM
            FETCH_BR_FIN   [BOOL] > VERIFICA AS INFORMAÇÕES PRESENTES NA LIB BR_FINANCE
            RECORD_FILE    [STR]  > ARQUIVO DE REFERENCIA DOS DADOS DO RED/CVM
            MERGING_FILES  [BOOL] > CONCATENAR INFORMACOES CVM - CNPJ - INFOS ADICIONAIS
            OUTPUT_MERGING [STR]  > ARQUIVO DE SAIDA COM TODAS AS INFORMACOES NECESSARIAS DO ATIVO
        """

        self.logger.info(f"CVM.FirmsInformations() > Extract_Data [Bool]: {extract_data}")
        self.logger.info(f"CVM.FirmsInformations() > Fetch_Data [Bool]: {fetch_data}")
        self.logger.info(f"CVM.FirmsInformations() > Fetch_Br_Fin [Bool]: {fetch_br_fin}")
        self.logger.info(f"CVM.FirmsInformations() > Record_File [Str]: {record_file}")
        self.logger.info(f"CVM.FirmsInformations() > Merging_Files [Str]: {merging_files}")
        self.logger.info(f"CVM.FirmsInformations() > Output_Merging [Str]: {output_merging}")

        if extract_data:
            self.logger.info(f"CVM.FirmsInformations() > Iniciando processo de AQUISICAO DE DADOS")

            # INFORMAÇÕES SOBRE AS COMPANHIAS BRASILEIRAS
            infos = []

            # TODAS AS LETRAS MAIUSCULAS E NUMEROS
            # CONFORME MOSTRADO NO BUSCADOR DA CVM
            capital_letters_and_numbers = []

            # TODAS AS LETRAS MAIUSCULAS VIA ASCII
            for ascii_value in range(ord('A'), ord('Z') + 1):
                capital_letters_and_numbers.append(chr(ascii_value))

            # TODOS OS DIGITOS POSSIVEIS
            for digit in range(10):
                capital_letters_and_numbers.append(str(digit))

            # PARA CADA POSSIBILIDADE NAS LISTAS CRIADAS
            # TENTAMOS ACESSAR AS TABELAS DISPONIVEIS
            for focus in capital_letters_and_numbers:

                # USA O SITE BASE DA CVM E ALTERA A LETRA INICIAL PARA BUSCAR AS TABELAS
                url = f'https://cvmweb.cvm.gov.br//SWB//Sistemas//SCW//CPublica//CiaAb//FormBuscaCiaAbOrdAlf.aspx?LetraInicial={focus}'
                self.logger.info(f"CVM.FirmsInformations() > Buscando empresas em {focus}")

                # USA O REQUEST PARA ACESSAR A URL
                response = requests.get(url)

                # SE O STATUS CODE = 200, ACESSO FOI UM SUCESSO
                if response.status_code == 200:

                    try:

                        # REALIZA O PARSING DO HTML VIA BEAUTIFULSOUP
                        html_content = response.text
                        soup = BeautifulSoup(html_content, 'html.parser')
                        # TODAS AS TABELAS NO HTML
                        tables = soup.find_all('table')

                        for table in tables:

                            # PROCESSAMOS OS ELEMENTOS 'TABELA'
                            rows = table.find_all('tr')

                            for row in rows:

                                # CÉLULAS DA TABELA
                                cells = row.find_all(['th', 'td'])
                                # TEXTO COMPLETO DE INFORMAÇÕES A SER EXTRAÍDO
                                full_txt = ""

                                for cell in cells:

                                    # ADQUIRE O TEXTO, SEM ESPAÇOS
                                    txt = cell.get_text(strip = True)

                                    if not txt.startswith(("CNPJ", "Selecione", " ")):
                                        # NÃO DESEJAMOS OS CABEÇALHOS PADRÃO
                                        full_txt += txt + ";"
                                    else: break

                                # CASO O TEXTO SEJA VALIDO, ACRESCENTA O ARQUIVO EMPRESAS BRASIL
                                if len(full_txt) > 0:
                                    infos.append(full_txt)                     

                    except Exception as exc:
                        # NOS AJUDA A COMPREENDER SE HOUVE UM ERRO E PORQUE
                        self.logger.error(f"CVM.FirmsInformations() > Excessao: {exc} em {focus}")

                else:
                    # NÃO FOI POSSIVEL REALIZAR O REQUEST
                    self.logger.error(f"CVM.FirmsInformations() > Problema Request: {response.status_code} em {focus}")

            # CRIA O ARQUIVO FINAL DE INFORMAÇÕES
            with open(record_file + ".txt", "w+") as record:
                for info in infos:
                    try:
                        self.logger.info(f"CVM.FirmsInformations() > Adicionando info {info} em {record_file}")
                        record.write(info + "\n")
                    except Exception as exc:
                        self.logger.error(f"CVM.FirmsInformations() > Problemas adicionando info {info} em {record_file}, {exc}")

        if fetch_data:

            def format_number(number):
                # MESMA REFERÊNCIA DE FORMATO UTILIZADA PELO BrFINANCE
                return f'{number:06d}'

            self.logger.info(f"CVM.FirmsInformations() > Iniciando processo de AJUSTE DE DADOS CVM")
            # EXTRAI O DATAFRAME DE INFORMAÇÕES DA CVM
            df = pandas.read_csv(
                record_file + ".txt", sep = ";", encoding = "latin1", header = None
            ).dropna(axis = 1)
            # NOME PADRÃO DAS COLUNAS
            df.columns = ["CNPJ", "FIRM_NAME", "CVM_FIRM_TYPE", "CVM_CODE", "CVM_STATUS"]
            # CORRIGE OS CÓDIGOS
            df["CVM_CODE"] = df["CVM_CODE"].apply(format_number)
            # SOBSCREVE AS INFORMAÇÕES
            df.to_csv(record_file + ".txt", index = False)
            self.logger.info(f"CVM.FirmsInformations() > Arquivo {record_file} sobreescrito")

        if fetch_br_fin:
            CVM.get_cvm_codes(self)
            CVM.get_cvm_categories(self)

        if merging_files:

            def for_col(df):
                x = pandas.DataFrame("-", index = ["x"], columns = df.columns)
                for col in df.columns:
                    y = df[col].unique().tolist()
                    if len(y) == 1: 
                        x.loc["x",col] = y[0]
                    else:
                        x.loc["x",col] = ';'.join(y)
                return(x)

            self.logger.info(f"CVM.FirmsInformations() > Iniciando processo de MERGING DE INFORMACOES")
            # DADOS DE PROVEDOR
            self.logger.info(f"CVM.FirmsInformations() > Adquirindo dados relacionais de provedor")
            data_provedor = pandas.read_csv("feeder_db//informacoes_acoes_brasil.csv", encoding = "utf-8", sep = ";", dtype=str)
            data_provedor["CNPJ"] = data_provedor["CNPJ"].str.replace(",", "", regex = False)
            data_provedor["CNPJ"][data_provedor["CNPJ"] != "-"] = data_provedor["CNPJ"].str.zfill(14)
            data_provedor["CNPJ"] = data_provedor["CNPJ"].astype(str).apply(CVM.format_cnpj)
            data_provedor.drop(["Nome.1"], axis = 1, inplace = True)
            data_provedor.columns = ["NAME", "CLASS", "CODE", "CNPJ", "STOCK_TYPE", "ISIN", "SETOR(BOVESPA)", "SUBSETOR(BOVESPA)", "SEGMENTO(BOVESPA)"]
            data_provedor = data_provedor[~(data_provedor.CODE.str.strip().str.contains(" "))]
            data_provedor.CNPJ[(data_provedor.CNPJ == "-")] = data_provedor.CODE
            #data_provedor = data_provedor.map(lambda x: x.upper() if isinstance(x, str) else x)
            for c in data_provedor.columns: data_provedor[c] = data_provedor[c].astype(str).str.upper()

            dataset = pandas.DataFrame("-", index = data_provedor.CNPJ.unique(), columns = data_provedor.columns)
            for cnpj in dataset.index:
                aux_df = data_provedor[data_provedor.CNPJ == cnpj]
                if aux_df.shape[0] == 1:
                    dataset.loc[cnpj,:] = aux_df.iloc[0,:].tolist()
                else:
                    dataset.loc[cnpj,:] = for_col(df = aux_df).iloc[0,:].tolist()
            # DADOS RAD/CVM
            self.logger.info(f"CVM.FirmsInformations() > Adquirindo dados relacionais da CVM")
            empresas_cvm = pandas.read_csv("feeder_db//EmpresasBrasilCVM.txt")
            empresas_cvm["CVM_CODE"] = empresas_cvm["CVM_CODE"].astype(str).str.zfill(6)
            empresas_cvm["SOURCE_CVM"] = True

            # DADOS RAD/CVM
            self.logger.info(f"CVM.FirmsInformations() > Adquirindo dados relacionais via BrFin")
            brfin = pandas.read_csv("feeder_db//CodigosDocsCVM.txt")
            brfin["CVMCode"] = brfin["CVMCode"].astype(str).str.zfill(6)
            brfin.drop(["CVMStatus"], axis = 1, inplace = True)
            brfin.columns = ["CVM_CODE", "FIRM_NAME"]
            brfin["SOURCE_BRFIN"] = True

            df = pandas.concat([empresas_cvm, brfin[~brfin.CVM_CODE.isin(empresas_cvm.CVM_CODE)]], axis = 0)
            df.SOURCE_BRFIN[(df.CVM_CODE.isin(brfin.CVM_CODE)) & (pandas.isna(df.CVM_CODE))] = True
            df.SOURCE_CVM[pandas.isna(df["SOURCE_CVM"])] = False
            df.SOURCE_BRFIN[pandas.isna(df["SOURCE_BRFIN"])] = False
            for c in df.columns:
                df[c][pandas.isna(df[c])] = "-"

            data = dataset.copy()
            data.index = dataset.CNPJ

            data["CVM_CODE"] = "-"
            data["CVM_STATUS"] = "-"
            data["CVM_FIRM_TYPE"] = "-"
            data["FIRM_NAME"] = "-"
            data["SOURCE_ECONOMATICA"] = True
            data["SOURCE_CVM"] = False
            data["SOURCE_BRFIN"] = False

            fetch_cols = ["CVM_CODE", "CVM_STATUS", "CVM_FIRM_TYPE", "FIRM_NAME", "SOURCE_CVM", "SOURCE_BRFIN"]

            df_in = df[(df.CNPJ.isin(data.index)) & (df.CNPJ != "-")]
            df_in.index = df_in.CNPJ

            data.loc[df_in.index, fetch_cols] = df_in[fetch_cols].values
            data.to_csv(output_merging + ".txt", index = False)
            self.logger.info(f"CVM.FirmsInformations() > Base de dados geral gerada em {output_merging}")

    def get_selected_firms(self):

        df = pandas.read_csv("feeder_db//Brazil.txt")
        df = df[(df.CNPJ != "-") & (df.CVM_CODE != "-")]
        self.firms = df.copy()
    
    def list_documents_cvm(self, search = [], start_date = date(1995,12,31), from_tickers = False):

        # EMPRESAS DE INTERESSE
        CVM.get_selected_firms(self)
        self.logger.info(f"CVM.ListDocumentsCVM() > Listagem de documentos das empresas da bolsa")
        # CATEGORIAS DE DOCUMENTOS DE INTERESSE
        categories = pandas.read_csv("feeder_db//CategoriasDocsCVM.txt")["BrFin"].tolist()
        self.logger.info(f"CVM.ListDocumentsCVM() > Total de categorias de documentos analisadas: {len(categories)}")
        
        # PASTA DE REFERENCIA DE DOCUMENTOS CVM
        if not os.path.isdir("cvm//"):
            os.mkdir("cvm//")
            existing_files = []
            self.logger.info(f"CVM.ListDocumentsCVM() > Diretorio 'cvm' criado")
        else:
            existing_files = os.listdir("cvm//")

        # CONTROLE DE BUSCA
        if len(search) == 0: search = self.firms["CVM_CODE"].unique().tolist()
        elif from_tickers: 
            search = CVM.get_data_from_tickers(ticker_list = search, data_type = "CVM_CODE", keep = False)
            search = sorted(list(set(search)))

        for cvm_code in search:
            # PARA CADA EMPRESA NA ANALISE
            got = False
            retries = 0
            # LACO DE RETRIES (TENTA EXTRAIR MAIS DE UMA VEZ OS RESULTADOS)
            while not got:
                try:
                    if retries < 3:
                        # EXTRACAO DOS DADOS DE INTERESSE
                        # REFERENCIA DE DOCUMENTOS
                        search_result = self.cvm_http_client.get_consulta_externa_cvm_results(
                            start_date = start_date,
                            end_date = date.today(),
                            cod_cvm = [cvm_code],
                            participant_type = [1],
                            category = categories,
                            last_ref_date = False
                        )
                        if f"Docs_CVM_{cvm_code}.txt" in existing_files:
                            # INFORMACOES JA EXISTENTES ACERCA DO ATIVO
                            previous_infos = pandas.read_csv(f"cvm//Docs_CVM_{cvm_code}.txt")
                            # MERGING
                            search_result = pandas.concat([
                                previous_infos,
                                search_result
                            ], axis = 0)
                            search_result.drop_duplicates(inplace = True)
                        # ADICIONA OS ARQUIVOS NA PASTA CVM
                        search_result.to_csv(f"cvm//Docs_CVM_{cvm_code}.txt")
                        self.logger.info(f"CVM.ListDocumentsCVM() > Dados armazenados com sucesso - Docs_CVM_{cvm_code}.txt")
                    # ENCERRA O LACO
                    got = True
                except Exception as exc:
                    # ERRO NA EXTRACAO DOS DADOS, TENTAMOS MAIS N VEZES
                    retries += 1
                    self.logger.warning(f"CVM.ListDocumentsCVM() > Problemas na extracao, codigo {cvm_code} - Tentativa: {retries} - Erro: {exc}")


    def get_reports(self, search = [], to_folder = "reports_cvm//", from_tickers = False):

        def append_file(df, file_name = ""):
            if os.path.isfile(file_name):
                #old_df = pandas.read_csv(file_name)
                #old_df.reset_index(drop = True, inplace = True)
                df = pandas.concat([
                    pandas.read_csv(file_name),
                    df
                ], axis = 0)
                df.drop_duplicates(inplace = True)
                df.to_csv(file_name, index = False)
            else:
                df.to_csv(file_name, index = False)

        # REPORTS DE INTERESSE
        reports_list = [
            'Balanço Patrimonial Ativo',
            'Balanço Patrimonial Passivo',
            'Demonstração do Resultado',
            'Demonstração do Resultado Abrangente',
            'Demonstração do Fluxo de Caixa',
            'Demonstração das Mutações do Patrimônio Líquido',
            'Demonstração de Valor Adicionado']
        
        # SIGLAS PARA OS REPORTS DE INTERESSE
        reports_list_resume = [
            'BALANCO_PATRIMONIAL_ATIVO', 
            'BALANCO_PATRIMONIAL_PASSIVO', 
            'DEMONSTRACAO_RESULTADO', 
            'DEMONSTRACAO_RESULTADO_ABRANGENTE', 
            'DEMONSTRACAO_FLUXO_CAIXA', 
            'DEMONSTRACAO_MUTUACOES_PATRIMONIO_LIQUIDO', 
            'DEMONSTRACAO_VALOR_ADICIONADO'
        ]

        # EMPRESAS DE INTERESSE
        CVM.get_selected_firms(self)
        self.logger.info(f"CVM.GetReports() > Extraindo relatorios estruturados CVM")

        # PASTAS DE REFERENCIA PARA OS ATIVOS
        if not os.path.isdir(to_folder):
            os.mkdir(to_folder)
            self.logger.info(f"CVM.GetReports() > Criando pasta {to_folder}")
            for report_type in reports_list_resume:
                os.mkdir(f"{to_folder}{report_type}//")
                self.logger.info(f"CVM.GetReports() > Criando pasta {to_folder}{report_type}")

        # ARQUIVO DE CONTROLE - CONTROLA OS ARQUIVOS EM CIRCULACAO
        file_controle = f"DocsList.txt"

        if file_controle in os.listdir(to_folder):
            # LEITURA DO CONTROLE DE DOCUMENTOS
            master = pandas.read_csv(f"{to_folder}{file_controle}", dtype = object)
            self.logger.info(f"CVM.GetReports() > Leitura do arquivo de referencia/controle")
        else:
            # NAO HA DOCUMENTO DE CONTROLE
            master = pandas.DataFrame(columns = ["DOC_ID"])
            self.logger.info(f"CVM.GetReports() > Arquivo de controle/referencia inexistente")

        # CONTROLE DE BUSCA
        if len(search) == 0: search = self.firms["CVM_CODE"].unique().tolist()
        elif from_tickers: 
            search = CVM.get_data_from_tickers(ticker_list = search, data_type = "CVM_CODE", keep = False)
            search = sorted(list(set(search)))
        # MAIS INTUITIVO ACOMPANHAR POR EMPRESA
        firms = CVM.get_data_from_cvm_code(cvm_codes = search, data_type = "FIRM_NAME", keep = True)

        # PARA CADA EMPRESA EM NOSSA LISTA
        for src in range(len(search)):
            cvm_code = search[src]
            frm_code = firms[src]
            print(f"Processo: {src + 1}/{len(search)}, Processando empresa: {frm_code}, CVM: {cvm_code}")

            self.logger.info(f"CVM.GetReports() > Analisando empresa {frm_code}, CVM: {cvm_code}")
            
            # ARQUIVO BR_FINANCE - LISTA TODOS OS DOCUMENTOS EXISTENTES
            file_brfinance = f"cvm//Docs_CVM_{cvm_code}.txt"

            try:
                # LEITURA DOS DOCUMENTOS
                documents = pandas.read_csv(file_brfinance, index_col = [0], dtype = object)
                # AJUSTE DO DATAFRAME DOS DOCUMENTOS
                documents = documents[pandas.to_numeric(documents['numero_seq_documento'], errors = 'coerce').notnull()]
                self.logger.info(f"CVM.GetReports() > Leitura dos dados {to_folder}")
                go_on = True

            except FileNotFoundError:
                # NAO HA DOCUMENTO CVM DE REFERENCIA DE ARQUIVOS
                self.logger.info(f"CVM.GetReports() > Arquivo de referencia {to_folder} nao existe, empresa {frm_code}, referencia {cvm_code}")
                go_on = False

            if go_on:

                for _, row in documents.iterrows():

                    # PARA TODOS OS CASOS VALIDOS, QUAL A CATEGORIA DO REPORT?
                    # A CATEGORIA NOS POLPA ESFORCOS NA BUSCA DOS DOCUMENTOS
                    categoria = row.categoria.split(" ")[0]

                    try:
                        if (categoria in ["DFP", "ITR"]):

                            # DOCUMENTO UNIQUE ID
                            docu_id = f"{cvm_code}-{categoria}-{row.ref_date}-V{row.version}-{row.numero_seq_documento}"

                            # VERIFICANDO SE O ARQUIVO JA NA EXISTE ou ALGUM ARQUIVO FOI MAL EXTRAIDO
                            if not docu_id in master.DOC_ID.tolist():

                                # REALIZA A AQUISICAO DOS REPORTS DAS EMPRESAS VIA CVM
                                reports = self.cvm_http_client.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list = reports_list)
                                # LISTA DE REPORTS DISPONIVEIS
                                key_list = list(reports.keys())
                                # DESCRITIVO DE REFERENCIA DO DOCUMENTO
                                row_description = pandas.DataFrame(row).transpose()

                            else: key_list = []

                        else: key_list = []

                    except Exception: key_list = [] # EM CASO DE ERRO NAO HA LISTA
                        
                    if len(key_list) > 0:

                        for key in key_list:
                            # NOME DO ARQUIVO DE REFERENCIA
                            docu_file = f"{to_folder}{reports_list_resume[reports_list.index(key)]}//{categoria}-{cvm_code}.txt"
                            #docu_rept = f"{docu_id}-{reports_list_resume[reports_list.index(key)]}"
                            
                            df = reports[key].copy()

                            if key != "Demonstração das Mutações do Patrimônio Líquido":
                                try:
                                    # CASO PADRAO DE REPORT, PROCESSAMENTO SIMPLES
                                    df["x"] = df["Valor"].astype(str) + " " + df["currency_unit"]
                                    # ADICIONAR A REFERENCIA DO CONTA A DESCRICAO SE MOSTROU VITAL EM DESAMBIGUAR
                                    df["Descrição"] = "(" + df["Conta"] + ") " + df["Descrição"]
                                    # PADRONIZA OS NANs
                                    df["x"][df.x.str.startswith("nan ")] = "-"
                                    # O INDICE E A PROPRIA DESCRICAO
                                    df.index = df["Descrição"]
                                    # TRANSFORMA EM LINHA
                                    df = df[["x"]].transpose()
                                    # PADRONIZA UM DUMMY INDEX
                                    row_description.index = ["x"]
                                    # VETOR LINHA A SER ADICIONADO
                                    df = pandas.concat([
                                        pandas.DataFrame([docu_id], columns = ["id"], index = ['x']), row_description, df], 
                                        axis = 1
                                    )
                                    discard_data = False
                                except Exception as exc:
                                    # ARQUIVO CORROMPIDO
                                    discard_data = True
                                    self.logger.error(f"CVM.GetReports() > Arquivo {docu_file} com problemas, ERRO: {exc}, ID: {docu_id}, CVM: {cvm_code}")
                            else:
                                try:
                                    # CASO MATRICIAL DISTINTO, MANTEM A REFERENCIA DE INDICE
                                    df.index = "(" + df["Conta"] + ") " + df["Descrição"]
                                    # TIPAGEM STRING/CHAR
                                    df = df.astype(str)

                                    for c in df.columns:
                                        # CADA COLUNA E PADRONIZADA EM CURRENCY
                                        if c not in ["Descrição", "currency_unit"]:
                                            df[c] = df[c] + " " + df["currency_unit"]

                                    # REDUZ O TAMANHO DO DF COM INFORMACOES NAO NECESSARIAS
                                    df.drop(["Descrição", "currency_unit", "Conta"], axis = 1, inplace = True)
                                    # DATAFRAME AUXILIAR
                                    aux = pandas.DataFrame()

                                    for c in df.columns:
                                        # FORMATO PADRAO DE NANs NO DATAFRAME
                                        df[c][df[c].str.startswith("nan ")] = "-"
                                        aux_c = df[[c]]
                                        # INDEX vs COLUNA
                                        aux_c.index = [i + " | " + c for i in aux_c.index]
                                        # INDEX PADRAO
                                        aux_c.columns = ["x"]
                                        # ACUMULA O RESULTADO
                                        aux = pandas.concat([aux, aux_c], axis = 0)

                                    # VETOR LINHAS A SER ADICIONADO
                                    df = pandas.concat([
                                        pandas.DataFrame([docu_id], columns = ["id"], index = ['x']), row_description, aux.transpose().copy()], 
                                        axis = 1
                                    )
                                    discard_data = False
                                except Exception as exc:
                                    # ARQUIVO CORROMPIDO
                                    discard_data = True
                                    self.logger.error(f"CVM.GetReports() > Arquivo {docu_file} com problemas, ERRO: {exc}, ID: {docu_id}, CVM: {cvm_code}")

                            if not discard_data:

                                docu_infos = pandas.DataFrame(
                                    [docu_id], 
                                    index = ["DOC_ID"],
                                    columns = ["X"]
                                ).transpose()

                                try:

                                    append_file(df = df, file_name = f"{docu_file}")
                                    self.logger.info(f"CVM.GetReports() > Arquivo {docu_file} atualzado")

                                    append_file(df = docu_infos, file_name = f"{to_folder}{file_controle}")
                                    self.logger.info(f"CVM.GetReports() > Atualizacao do controle {to_folder}{file_controle} para {cvm_code}")

                                except Exception as exc:
                                    self.logger.error(f"CVM.GetReports() > {exc} - {docu_file} para {cvm_code}")

    def get_data_from_cvm_code(cvm_codes = [], data_type:str = "FIRM_NAME", keep = False):
        if type(cvm_codes) == str: cvm_codes = [cvm_codes]
        # CONTROLE GERAL DE TICKERS E DADOS
        df = pandas.read_csv("feeder_db//Brazil.txt", sep = ",")
        codes = []
        for cvmc in cvm_codes:
            aux = df[df.CVM_CODE == cvmc]
            if aux.shape[0] > 0: codes.append(aux[[data_type]].iloc[0, 0])
            elif keep: codes.append(-1)
        return codes

    def get_data_from_tickers(ticker_list = [], data_type:str = "CVM_CODE", keep = False):

        """ GET_DATA_FROM_TICKERS()

            TICKER_LIST [STR/LIST]: TICKERS DE INTERESSE
            DATA_TYPE [STR]: DADO DESEJADO
            KEEP: MANTEM A ORDEM DOS DADOS (INCLUI DADOS NAO ENCONTRADOS)
        """

        def find_position_by_ticker(x, ticker_list):
            # CASO QUEIRA APENAS UM TICKER
            if type(ticker_list) == str: ticker_list = [ticker_list]
            # EXPANDE OS POSSIVEIS TICKERS
            finder = x["CODE"].str.split(";", expand = True)
            finder.index = x.index
            # RESULTADOS ESPERADOS
            pos = []
            for ticker in ticker_list:
                # VARIAVEL DE CONTROLE
                gotcha = False
                for c in finder.columns:
                    # EXISTE UMA COLUNA COM TAL TICKER?
                    aux = finder[c][finder[c] == ticker]
                    if aux.shape[0] > 0:
                        # CASO POSITIVO
                        pos.append(aux.index.tolist()[0])
                        gotcha = True
                # CASO NAO TENHA SIDO POSSIVEL ENCONTRAR
                if not gotcha: pos.append(-1)
            return pos

        # CONTROLE GERAL DE TICKERS E DADOS
        df = pandas.read_csv("feeder_db//Brazil.txt", sep = ",")
        # POSICOES ENCONTRADAS
        positions = find_position_by_ticker(x = df, ticker_list = ticker_list)

        codes = []
        for t in positions:
            # EXTRACAO DO DADO DE INTERESSE
            if t != -1: codes.append(df[[data_type]].iloc[t, 0])
            elif keep: codes.append(-1)
        return codes

In [ ]:
cvm = CVM()

""" EXTRACAO DE INFORMACOES IMPORTANTES PARA A EXECUCAO DO PROGRAMA

    EXTRACT_DATA   - SE TRUE, REALIZA O SCRAPING DOS DADOS RAD/CVM
    FETCH_DATA     - SE TRUE, MODIFICACOES NA BASE DE DADOS RAD/CVM (EM PROCESSO DE DESCONTINUACAO)
    FETCH_BR_FIN   - SE TRUE, ADQUIRE INFORMACOES IMPORTANTES VIA BrFINANCE
    RECORD_FILE    - ARQUIVO DE REFERENCIA PARA OS DADOS RAD/CVM
    MERGING_FILES  - SE TRUE, REALIZA O MERGING DE TODOS OS ARQUIVOS
    OUTPUT_MERGING - ARQUIVO DE DADOS CONCATENADOS  
"""

cvm.firms_informations(
    extract_data = False, 
    fetch_data = False, 
    fetch_br_fin = False, 
    record_file = "feeder_db//EmpresasBrasilCVM", 
    merging_files = False, 
    output_merging = "feeder_db//Brazil"
)

# LISTA DE ATIVOS DE INTERESSE
search = ["RRRP3"]

In [ ]:
""" REFERENCIA DE DOCUMENTOS CVM

    EXTRAI TODAS AS REFERENCIAS DE DOCUMENTOS CONTIDAS NA CVM PARA EXTRACAO
"""

# TODO FEED CONTINUO QUE EVITA LONGA PESQUISA DE DOCUMENTOS
cvm.list_documents_cvm(search = search, from_tickers = True)

In [ ]:
""" EXTRACAO DOS RELATORIOS ESTRUTURADOS

    APENAS DADOS ESTRUTURADOS VIA MODIFICACAO NO BrFINANCE
"""

cvm.get_reports(search = search, from_tickers = True)